In [1]:
import torch
import torch.nn as nn
import os
import sys
from diffusers.utils import export_to_video
sys.path.append('/root/SVD/src')
from transformers import CLIPTextModel, CLIPTokenizer, CLIPModel, CLIPImageProcessor
from diffuser.models.unet_action_v11 import UNetSpatioTemporalConditionModel_Action
from diffuser import StableDiffusionPipeline, AutoencoderKL, DDIMScheduler, UNet2DConditionModel
from safetensors import safe_open
from diffuser.pipelines.stable_video_diffusion.pipeline_action_video_diffusion_v1_v_imclip import ActionVideoDiffusionPipeline

2024-08-07 12:13:19.548362: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-07 12:13:19.747708: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-07 12:13:20.344262: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-07 12:13:23.383512: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/root/SVD/src/diffuser/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is depr

In [2]:
# prepare components
pretrained_model_name_or_path = "/mnt/storage/user/wangxiaodong/DWM_work_dir/lidar_maskgit_debug/smodels-vis/ti2v_s448_imclip_boxmap_allframe/checkpoint-44000"
device = 'cuda:0'
text_encoder = CLIPTextModel.from_pretrained(
    '/mnt/storage/user/wuzehuan/Downloads/models/stable-diffusion-2-1', subfolder="text_encoder", variant="fp16"
)
vae = AutoencoderKL.from_pretrained(
    '/mnt/storage/user/wuzehuan/Downloads/models/stable-diffusion-2-1', subfolder="vae"
)
clip_model = CLIPModel.from_pretrained(
    "/mnt/storage/user/wuzehuan/Downloads/models/CLIP-ViT-H-14-laion2B-s32B-b79K", torch_dtype=torch.float16)

unet = UNetSpatioTemporalConditionModel_Action(cross_attention_dim=1024, in_channels=4, temp_style = "image")
if "unet" not in os.listdir(pretrained_model_name_or_path):
    dm_path = os.path.join(pretrained_model_name_or_path, "diffusion_pytorch_model.safetensors")
else:
    dm_path = os.path.join(pretrained_model_name_or_path, "unet", "diffusion_pytorch_model.safetensors")
tensors = {}
with safe_open(dm_path, framework="pt", device='cpu') as f:
    for k in f.keys():
        tensors[k] = f.get_tensor(k)
miss_keys, ignore_keys = unet.load_state_dict(tensors, strict=True)
print(f'loaded unet from {pretrained_model_name_or_path}')
print('miss_keys: ', miss_keys)
print('ignore_keys: ', ignore_keys)
text_encoder.eval()
vae.eval()
clip_model.eval()
text_encoder.to(device)
vae.to(device)
clip_model.to(device)

tokenizer = CLIPTokenizer.from_pretrained('/mnt/storage/user/wuzehuan/Downloads/models/stable-diffusion-2-1', subfolder="tokenizer")
scheduler = DDIMScheduler.from_pretrained('/mnt/storage/user/wuzehuan/Downloads/models/stable-diffusion-2-1', subfolder="scheduler")
feature_extractor = CLIPImageProcessor.from_pretrained('/mnt/storage/user/wuzehuan/Downloads/models/stable-diffusion-2-1', subfolder="feature_extractor")


add image context parameters
down & mid context in channel: 320
cross_attention_dim: 1024
init temp style: image
cross_attention_dim: 1024
init temp style: image
add image context parameters
down & mid context in channel: 320
cross_attention_dim: 1024
init temp style: image
cross_attention_dim: 1024
init temp style: image
add image context parameters
down & mid context in channel: 640
cross_attention_dim: 1024
init temp style: image
cross_attention_dim: 1024
init temp style: image
add image context parameters
down & mid context in channel: 1280
cross_attention_dim: 1024
init temp style: image
add image context parameters
down & mid context in channel: 1280
add image context parameters
up block context in channels: 1280
add image context parameters
up block context in channels: 640
cross_attention_dim: 1024
init temp style: image
cross_attention_dim: 1024
init temp style: image
cross_attention_dim: 1024
init temp style: image
add image context parameters
up block context in channels: 32

In [3]:
unet.eval()
unet = unet.to(device)
pipeline = ActionVideoDiffusionPipeline(
        text_encoder=text_encoder,
        vae=vae,
        unet=unet,
        scheduler=scheduler,
        tokenizer=tokenizer,
        feature_extractor=feature_extractor,
        clip_model=clip_model
)
pipeline = pipeline.to(device).to(torch.float16)

scheduler prediction type: v_prediction


In [4]:
import timm
layout_encoder = timm.create_model(model_name="convnextv2_base.fcmae", pretrained=True, num_classes=0)
layout_encoder.to(device)
layout_encoder = layout_encoder.half()

In [5]:
# image processer
from torchvision import transforms
from PIL import Image
transform = transforms.Compose([
    transforms.Resize((256, 448)),
    transforms.ToTensor()
])

In [6]:
utime = f'{1532402941762460}'

In [7]:
_3dbox_image_path = f"/mnt/storage/user/wangxiaodong/DWM_work_dir/lidar_maskgit_debug/mini_3dbox_hdmap/3dbox_{utime}.jpg"
hdmap_image_path = f"/mnt/storage/user/wangxiaodong/DWM_work_dir/lidar_maskgit_debug/mini_3dbox_hdmap/hdmap_{utime}.jpg"
_3dbox_image = Image.open(_3dbox_image_path)
hdmap_image = Image.open(hdmap_image_path)
_3dbox = transform(_3dbox_image)
hdmap = transform(hdmap_image)
_3dbox = _3dbox.unsqueeze(0).to(device)
hdmap = hdmap.unsqueeze(0).to(device)

In [8]:
image_path = f"/mnt/storage/user/wangxiaodong/DWM_work_dir/lidar_maskgit_debug/mini_3dbox_hdmap/{utime}.jpg"
image = Image.open(image_path).convert("RGB")
with open(f"/mnt/storage/user/wangxiaodong/DWM_work_dir/lidar_maskgit_debug/mini_3dbox_hdmap/caption_{utime}.txt", 'r', encoding='utf-8') as file:
    caption = file.read()

In [9]:
weight_dtype = torch.float16
_3dbox_embeddings = layout_encoder\
    .forward_features(_3dbox.to(weight_dtype))\
    .flatten(-2).permute(0, 2, 1)
hdmap_embeddings = layout_encoder\
    .forward_features(hdmap.to(weight_dtype))\
    .flatten(-2).permute(0, 2, 1)
hdmap_embeddings.shape

torch.Size([1, 112, 1024])

In [10]:
# provide prompt_embeds
print(caption)
inputs = tokenizer(
    caption, max_length=tokenizer.model_max_length, padding="max_length", truncation=True, return_tensors="pt"
    )
text_input = inputs['input_ids'].to(device)
encoder_hidden_states = text_encoder(text_input)[0]
encoder_hidden_states.shape

the car is parked on the road.


torch.Size([1, 77, 1024])

In [11]:
# final encoder hidden states
encoder_hidden_states = torch.cat([encoder_hidden_states, _3dbox_embeddings, hdmap_embeddings], dim=1)

In [12]:
video = pipeline(image=image, num_frames=8, action=None, prompt_embeds=encoder_hidden_states, height=256, width=448).frames[0]

image clip embedding torch.Size([1, 1, 1024])
text emb shape:  torch.Size([1, 301, 1024])
no noise add to image


  0%|          | 0/25 [00:00<?, ?it/s]

In [13]:
export_to_video(video, os.path.join('/mnt/storage/user/wangxiaodong/DWM_work_dir/lidar_maskgit_debug', f'utime_{utime}.mp4'), fps=6)

'/mnt/storage/user/wangxiaodong/DWM_work_dir/lidar_maskgit_debug/utime_1532402941762460.mp4'